# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [7]:
# Connect to SQLite database.

conn = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')
conn
# Create a cursor object
cursor = conn.cursor()

In [8]:
# List all the tables in the database.
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the list of table names
print("Tables in the database:")
for table in tables:
    print(table[0])


Tables in the database:
stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [9]:
# Write the query to be executed that selects everything from the online_retail_history table.

query = 'SELECT* FROM online_retail_history'

# Use the read_sql function in pandas to read a query into a DataFrame.

online_retail_history = pd.read_sql(query, conn)

# Preview the first five rows of the data.
online_retail_history.head(n=5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [10]:
# Get the shape of the data.

online_retail_history.shape

(15321, 8)

# Read data from the `stock_description` table

In [11]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT* FROM stock_description'


# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description = pd.read_sql(query, conn)

# Preview the first five rows of the data.

stock_description.head(n=3)

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER


In [12]:
# Get the shape of the data.

stock_description.shape

(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [13]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.

query = '''SELECT*
         FROM online_retail_history
         JOIN stock_description ON online_retail_history.StockCode = stock_description.StockCode'''




# Use the read_sql function in pandas to read a query into a DataFrame.

agg = pd.read_sql(query, conn)



# Preview the first five rows of the data.

agg.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [14]:
# Get the shape of the data.

agg.shape

(17032, 10)

# Identify and fix corrupt or unusable data

In [17]:
# Check the value counts of the "Description" field.
agg.Description.value_counts()


CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [21]:
# Remove rows where "Description" is just a question mark (?).

filtered_df = agg[agg['Description'] != '?']


# Preview the first five rows of the data.
filtered_df.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


# Identify and remove duplicates

In [22]:
# Identify all duplicated data.

duplicates = filtered_df[filtered_df.duplicated()]



In [23]:
# Print the duplicated data.

print("Duplicated data:")
print(duplicates)

Duplicated data:
      Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
178    536863     20727         1  2010-12-03 11:19:00   1.65     u17967   
497    537781     84879         8  2010-12-08 12:46:00   1.69     u17341   
571    537955     20725         1  2010-12-09 11:28:00   1.65     u16782   
935    539092     22423        16  2010-12-16 10:08:00  10.95     u15482   
1068   539475     22197         1  2010-12-19 14:41:00   0.85     u16686   
...       ...       ...       ...                  ...    ...        ...   
16542  580048     20727         1  2011-12-01 12:53:00   1.65     u12748   
16671  580469     20727         2  2011-12-04 12:32:00   1.65     u14583   
16673  580469     20727         1  2011-12-04 12:32:00   1.65     u14583   
16716  580611     20727         1  2011-12-05 11:49:00   1.65     u12748   
16996  581449     22423         1  2011-12-08 17:37:00  12.75     u12748   

              Country  TotalAmount StockCode                    Descri

In [26]:
# Remove the duplicated data.
df_without_duplicates = filtered_df.drop_duplicates()




# Preview the first five rows of the data.
df_without_duplicates.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


# Correct date formats

In [31]:
# Get the data types for every column in the DataFrame.
df_without_duplicates.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   StockCode    15206 non-null  object 
 9   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.3+ MB


In [32]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
df_without_duplicates['InvoiceDate']= pd.to_datetime(df_without_duplicates['InvoiceDate'],format ='%Y-%m-%d')



In [33]:
# Get the data types for every column in the converted DataFrame.
df_without_duplicates.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   StockCode    15206 non-null  object        
 9   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.3+ MB


# Examine the table before finishing

In [34]:
# Preview the first five rows of the data.
df_without_duplicates.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [35]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
df_without_duplicates.to_pickle('online_history_cleaned.pickle')


In [36]:
# Close any connections to the database.

conn.close()